In [782]:
import selenium.webdriver as webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import re
from collections import defaultdict
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from io import BytesIO
from io import StringIO
import subprocess
import shutil
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
import openai
from openai import OpenAI
import os

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from deutschland.bundesanzeiger import Bundesanzeiger


def get_chromium_version() -> str:
    try:
        result = subprocess.run(['chromium', '--version'], capture_output=True, text=True)
        version = result.stdout.split()[1]
        return version
    except Exception as e:
        return str(e)

def get_chromedriver_version() -> str:
    try:
        result = subprocess.run(['chromedriver', '--version'], capture_output=True, text=True)
        version = result.stdout.split()[1]
        return version
    except Exception as e:
        return str(e)

def get_chromedriver_path() -> str:
    return shutil.which('chromedriver')

def get_webdriver_options(headless=False):
    options = Options()
    
    if headless:
        # The newer headless argument is "--headless=new" for Chrome 109+
        # If you experience compatibility issues, fallback to "--headless"
        options.add_argument("--headless=new")

    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-features=NetworkService")
    options.add_argument("--window-size=1920x1080")
    options.add_argument("--disable-features=VizDisplayCompositor")
    options.add_argument('--ignore-certificate-errors')
    return options

def get_webdriver_service():
    service = Service(
        executable_path=get_chromedriver_path()
    )
    return service

def run_selenium(domain, headless=False):
    """
    Attempts to fetch the HTML content of `domain` using Selenium.
    If headless=True, runs in headless mode; otherwise, opens a visible browser window.
    """
    html_content = None

    options = get_webdriver_options(headless=headless)
    service = get_webdriver_service()

    with webdriver.Chrome(options=options, service=service) as driver:
        try:
            driver.get(domain)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            html_content = driver.page_source
        except Exception as e:
            print(f"Error occurred while fetching {domain} in headless={headless} mode: {e}")

    return html_content

def extract_body_content(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    body_content = soup.body
    if body_content:
        return str(body_content)
    return ""

def clean_body_content(body_content):
    soup = BeautifulSoup(body_content,'html.parser')

    for script_or_style in soup(['script', 'style']):
        script_or_style.extract()

    cleaned_content = soup.get_text(separator='\n')
    cleaned_content = '\n'.join(
        line.strip() for line in cleaned_content.splitlines() if line.strip()
        )
    return cleaned_content

def split_dom_content(dom_content, max_length = 8000):
    return [
        dom_content[i : i + max_length] for i in range(0,len(dom_content), max_length)
    ]

def is_valid_url(url):
    parsed = urlparse(url)
    return all([parsed.scheme, parsed.netloc])  # Checks for scheme (http/https) and netloc (domain)


def choose_links(links):
    print("Here are the links to explore:\n")
    for i, link in enumerate(links):
        print(f"{i}: {link}")
    
    # Example input: "0,2"
    user_input = input("\nSelect the indexes of links you want to explore, separated by commas:\n> ")
    
    # Convert the input string into a list of integers
    indexes = [int(x.strip()) for x in user_input.split(",") if x.strip().isdigit()]
    
    chosen_links = [links[i] for i in indexes if i < len(links)]
    return chosen_links

# Define the list of language codes
language_codes = [
    "en", "de", "fr", "es", "it", "pt", "nl", "ru", "zh", "ja", "ko", "ar", "hi", "bn", "ur", "id", "ms",
    "vi", "th", "pl", "tr", "sv", "no", "fi", "da", "cs", "hu", "ro", "el", "he", "sk", "bg", "uk", "sr",
    "hr", "sl", "lt", "lv", "et", "fa", "tl", "sw", "mt", "is", "ga", "cy", "sq", "mk"
]

# Define the function
def extract_second_level_links(links, base_url):
    second_level_links = []
    
    for link in links:
        parsed_url = urlparse(link)
        path_parts = parsed_url.path.strip('/').split('/')

        # Ensure the link is from the same domain and meets hierarchy criteria
        if len(path_parts) > 0 and len(path_parts) < 3 and parsed_url.netloc == urlparse(base_url).netloc:
            if path_parts[0] in language_codes and len(path_parts) > 1:
                # Language-specific link with subcategory
                full_link = f"{base_url}/{path_parts[0]}/{path_parts[1]}"
                second_level_links.append(full_link)
            elif path_parts[0] not in language_codes:
                # Link without language prefix
                full_link = f"{base_url}/{path_parts[0]}"
                second_level_links.append(full_link)
    
    # Remove duplicates
    second_level_links = list(set(second_level_links))
    
    # Create a DataFrame
    second_level_df = pd.DataFrame(second_level_links, columns=['links'])
    
    # Extract the topic from the URL
    second_level_df['topic'] = second_level_df['links'].apply(
        lambda x: urlparse(x).path.strip('/').split('/')[-1]
    )
    second_level_df.set_index(['topic'],inplace = True)
    return second_level_df

def get_all_links(domain):
    links_to_visit = []
    try:
        html = run_selenium(domain)

        # If run_selenium returned None, bail out early:
        if html is None:
            print(f"Error: No HTML returned from run_selenium for '{domain}'")
            return links_to_visit
        
        # save content
        body_content = extract_body_content(html)  # Extract body content
        cleaned_content = clean_body_content(body_content)  # Clean the content
        
        #pull respective links
        soup = BeautifulSoup(html, 'html.parser')
        for a_tag in soup.find_all('a', href=True):
            # Resolve relative URLs
            full_url = urljoin(domain, a_tag['href'])
            links_to_visit.append(full_url)
        

    except Exception as e:
        # Refer to the domain (which is always defined), not full_url
        print(f"Error visiting domain '{domain}': {e}")

    return links_to_visit, cleaned_content

def get_content_only(domain):
    try:
        html = run_selenium(domain)

        # If run_selenium returned None, bail out early:
        if html is None:
            print(f"Error: No HTML returned from run_selenium for '{domain}'")
            return links_to_visit
        
        # save content
        body_content = extract_body_content(html)  # Extract body content
        cleaned_content = clean_body_content(body_content)  # Clean the content

    except Exception as e:
        # Refer to the domain (which is always defined), not full_url
        print(f"Error visiting domain '{domain}': {e}")

    return cleaned_content

import re
import math
import os
from urllib.parse import urlparse

def filter_urls_by_stem(base_url, candidate_urls, max_depth=0):
    """
    Filters `candidate_urls` such that:
      1) They share the same domain as `base_url`.
      2) Their path depth does not exceed `base_depth + max_depth`.
      3) They are not exactly the base URL.

    :param base_url: The main starting URL (string).
    :param candidate_urls: A list of URLs discovered from the current page.
    :param max_depth: The maximum additional path segments allowed beyond the base URL’s depth.
    :return: A list of filtered URLs.
    """
    parsed_base = urlparse(base_url)
    base_domain = parsed_base.netloc
    # Remove leading/trailing slashes to avoid empty last segment
    base_path = parsed_base.path.strip("/")
    full_base_url = f'{base_domain}/{base_path}'
    base_depth = len(full_base_url.split("/")) if full_base_url else 0

    matches = []
    for url in candidate_urls:
        parsed = urlparse(url)
        if parsed.fragment:
            continue
        candidate_base = parsed.netloc
        candidate_path = parsed.path.strip("/")
        full_candidate_url = f'{candidate_base}/{candidate_path}'
        candidate_depth = len(full_candidate_url.split("/")) if full_candidate_url else 0
        # 1) Ensure same domain + that candidate url is a level down from base url
        if full_candidate_url.split('/')[:base_depth] == full_base_url.split("/") and candidate_depth > base_depth:
            if candidate_depth - base_depth <= (max_depth + 1):
                matches.append(url)

    return matches


from concurrent.futures import ThreadPoolExecutor, as_completed

def process_link(link, base_url, depth=0):
    """
    Processes a single link by:
      1) Getting sub_links and content
      2) Filtering based on the original base_url + max_depth
      3) Expanding up to 'depth' levels
      4) Returning (link, link_set, content_dict_sub)
    """
    # 1) Grab links & content for this page
    sub_links, sub_content = get_all_links(link)

    content_dict_sub = {}
    content_dict_sub[link] = sub_content

    link_set = set()

    # 2) First pass: filter sub_links by base_url
    first_pass = filter_urls_by_stem(base_url, sub_links, max_depth=depth)
    link_set.update(first_pass)

    frontier = set(first_pass)

    # 3) Expand up to specified depth
    for depth_level in range(depth):
        next_frontier = set()
        for current_link in frontier:
            current_links, current_content = get_all_links(current_link)
            content_dict_sub[current_link] = current_content

            new_links = set(filter_urls_by_stem(base_url, current_links, max_depth=depth))
            new_links -= link_set

            link_set.update(new_links)
            next_frontier.update(new_links)

        frontier = next_frontier
        
    for frontier_link in frontier:
        frontier_content = get_content_only(frontier_link)
        content_dict_sub[frontier_link] = frontier_content
        
    return (link, link_set, content_dict_sub)

def preprocess(chosen_links, depth=0):
    start_time = time.time()

    link_library = {}
    content_dict_master = {}

    max_workers = min(8, len(chosen_links))
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_link = {
            # Note how we pass `link` as both `link` and `base_url`
            executor.submit(process_link, link, link, depth): link
            for link in chosen_links
        }

        for future in as_completed(future_to_link):
            link = future_to_link[future]
            try:
                processed_link, link_set, content_dict_sub = future.result()
                link_library[processed_link] = list(link_set)
                content_dict_master[processed_link] = content_dict_sub
            except Exception as e:
                print(f"Error processing link {link}: {e}")

    end_time = time.time()
    print(f"\nPreprocess finished in {end_time - start_time:.2f} seconds")

    return link_library, content_dict_master

def preprocess_content(content_dict):
    master_dict = {}
    for key in content_dict.keys():
        master_dict[key] = ''.join(content_dict[key].values())
    return master_dict

def run_chatgpt(prompt, model = "gpt-3.5-turbo"):
    client = OpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    )
    response = client.chat.completions.create(
        model=model,
        messages=[
       {"role": "system", "content": "You are an advanced data extraction assistant."},
       {"role": "user", "content": prompt}
        ],
        temperature=0.0  # For deterministic responses
    )
    return response

def first_pass_with_chatGPT(dom_chunks):
    """
    Sends chunks of website content to OpenAI's GPT API for parsing based on a description.
    """

    # Loop through chunks and get responses
    parsed_results = []
    # Step 1: Analyze the first chunk with GPT-4o
    first_chunk = dom_chunks[0]
    context_prompt = (
        f"You are tasked with analyzing the following content:\n\n{first_chunk}\n\n"
        "1. Identify the type of business described in the content.\n"
        "2. Determine what is being described (e.g., business's products, infrastructure, locations, or verticals).\n"
        "3. Your output has to be a consise summary of points 1 and 2 in one sentence (not more than 15 words). And clear instructions on which table columns to use to summarise the information (not more than 5 columns)"
        " For example if its product information the columns should be 'product','description','target customers', 'KPIs', if its data centers"
        " the columns should be 'data center','location','Power size in MW', 'size in SQM','customers','expansion capacity' etc.")

    context = run_chatgpt(context_prompt,model = 'gpt-4o')
    context = context.choices[0].message.content

    print(context)

    for i, chunk in enumerate(dom_chunks, start=1):
        template = (
            f'You are tasked with extracting specific information from the following text content: {chunk}.'
            'Please follow these instructions carefully: \n\n'
            ' 1. ** Extract Information** : Only extract the information that directly matches the below description'
            ' 2. ** No Extra Content** : Do not include any additional text, comments or explanations in your response.'
            ' 3. ** Empty Response** : If no information matches the description return to an empty string'
            f' 4. ** Content instructions** : {context}.'
            ' 5. ** Format** : **Single Table**: Output exactly one table, no extra lines or text.'
                            '  - Use semicolons (`;`) to separate columns.'
                            '  - Properly quote cells with ("") if they contain semicolons or commas.'
                            '  - If a cell has no data, use `empty`.'
            )
        prompt = template
        print(f"Processing chunk {i}/{len(dom_chunks)}...")

        try:
            response = run_chatgpt(prompt)
            parsed_results.append(response.choices[0].message.content)

        except Exception as e:
            print(f"Error processing chunk {i}: {e}")
            parsed_results.append("")  # Append an empty string if there's an error

    initial_result = "\n".join(parsed_results)

    return initial_result

def parse_semicolon_csv(llm_output, delimiters = None):
    """
    Extracts table data from LLM output (potentially wrapped in ```csv code fences),
    dynamically handles multiple delimiters, and ensures the resulting DataFrame has multiple columns.
    
    :param llm_output: The text output from the LLM containing tabular data.
    :param delimiters: List of possible delimiters to handle, e.g., [",", ";", "\t", "|"].
                       Defaults to common delimiters.
    :return: A pandas DataFrame containing the parsed table data.
    """
    # Default delimiters if none are provided
    if delimiters is None:
        delimiters = [",", ";", "\t", "|"]
    
    # 1) Extract the CSV text from code fences, if present
    pattern = r"```csv\n(.*?)```"
    match = re.search(pattern, llm_output, flags=re.DOTALL | re.IGNORECASE)

    if match:
        csv_text = match.group(1).strip()
    else:
        # If no fenced block found, assume entire output is the CSV
        csv_text = llm_output.strip()

    # 2) Basic cleanup: remove trailing spaces or lines
    lines = [line.strip() for line in csv_text.splitlines() if line.strip()]
    csv_text = "\n".join(lines)

    # 3) Try parsing the CSV with each delimiter
    for delimiter in delimiters:
        try:
            # Attempt to parse the text into a DataFrame
            df = pd.read_csv(
                StringIO(csv_text),
                sep=delimiter,
                engine="python",
                quotechar='"',
                on_bad_lines="skip"  # Skip lines that don't match columns
            )
            # Check if the resulting DataFrame has more than one column
            if df.shape[1] > 1:
                break
        except Exception:
            # Ignore errors and try the next delimiter
            continue
    else:
        # If none of the delimiters work, raise an error
        raise ValueError(
            "Could not parse the table data with any of the specified delimiters or the result only has one column."
        )
    
    # 4) Remove empty and unnamed columns
    df.dropna(axis="columns", how="all", inplace=True)
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    # 5) Clean column names
    df.columns = [col.strip('" ').strip() for col in df.columns]

    # 6) Reset the index
    df.reset_index(drop=True, inplace=True)

    return df

def scrape_to_df(content_dict):
    start_time = time.time()
    outputs = {}
    master_dict = preprocess_content(content_dict)
    counter = 0
    for key in master_dict:
        counter += 1
        print(f'Processing {key}, {counter} out of {len(master_dict.keys())} datasets to process.')
        dom_content = master_dict[key]
        dom_chunks = split_dom_content(dom_content)
        csv_output = first_pass_with_chatGPT(dom_chunks)
        outputs[key] = parse_semicolon_csv(csv_output)
    end_time = time.time()
    print(f"\nParsing scrape results finished in {end_time - start_time:.2f} seconds")
    return outputs

def run_research(website,parse_description):
    links,content = preprocess(website)
    output = scrape_to_df(content,parse_description)
    return output

def select_topics():
    def is_valid_url(url):
        pattern = re.compile(r'^(https?:\/\/)?([\w.-]+)+(:\d+)?(\/[\w._-]*)*\/?$')
        return re.match(pattern, url)

    website_input = input("\nPlease paste the websites you'd like to explore, separated by commas: ")
    if not website_input.strip():
        print("No websites entered. Please try again.")
        return []

    websites = [str(x.strip().lower().rstrip("/")) for x in website_input.split(",") if x.strip()]
    valid_websites = [x for x in websites if is_valid_url(x)]

    if len(valid_websites) == 0:
        print("No valid websites provided.")
        return []

    print(f"Websites selected: {', '.join(valid_websites)}")
    return valid_websites

def user_selects(players = None):
    if players is None:
        domains = select_topics()
    else:
        domains = players.tolist() if isinstance(players, pd.Series) else players
    domain_links = {}
    for domain in domains:
        initial_links, initial_content = get_all_links(domain)
        domain_links[domain] = choose_links(initial_links)
    return domain_links

def user_task():
    user_input = input('\nPlease let DumbResearch know:\n1. What web content it is looking at \n2. What columns you would like your research output table to have \n\nSee example below: \n"These are the solutions offered by a german cyber security player (input is provided in German), please provide a table or tables that list the respective products / solutions with important information on the respective products e.g. their descriptions, features, target customers, KPIs"\n')
    return user_input

def initialize_dumbresearch(players = None):
    if players is not None and not isinstance(players, (list, pd.Series)):
        raise ValueError("`players` must be a list or Pandas Series of URLs")
    domains_selected = user_selects(players)
    return domains_selected

def dumbresearch_workflow(players = None):
    domains = initialize_dumbresearch(players)
    outputs = {}
    for key in domains.keys():
        links, content = preprocess(domains[key])
        output = scrape_to_df(content)
        outputs[key] = output
    return outputs

def find_competitors():
    company = input('\n**Target**\nPlease let DumbResearch know, on which company you are conducting desktop research')
    company_context = input('\n**Target description**\nPlease let DumbResearch know more about the company, i.e. what industry, geographies and if you want it to pay particular attention to anything or exclude something in its research!\n')
    result = parse_semicolon_csv(competitors_with_chatGPT(company,company_context))
    return result


def competitors_with_chatGPT(company, company_context):
    """
    Sends chunks of website content to OpenAI's GPT API for parsing based on a description.
    """

    # Loop through chunks and get responses
    parsed_results = []
    template = (
        f'You are tasked with finding the competitors of a given company called {company}, with the below context provided: {company_context}.'
        'Please follow these instructions carefully: \n\n'
        ' 1. ** Extract Information** : Only extract the information that directly matches the provided description above'
        ' 2. ** Volume** : Find at least 10 competitors for each requests.'
        ' 3. ** Empty Response** : If no information matches the description return to an empty string'
        ' 4. ** Direct Data** : Your response should contain only data that is explicitly requested, with no other text.'
        ' 5. ** Format** : **Single Table**: Output exactly one table, no extra lines or text.'
                        '  - Use semicolons (`;`) to separate columns.'
                        '  - Properly quote cells with ("") if they contain semicolons or commas.'
                        '  - If a cell has no data, use `empty`.'
                        '  - Columns are the 1) respective competitors ("Competitor") 2) the industry they are in ("Industry") 3) the link to their website ("Website") 4) The name of the legal entity that is registered with the authorities ("Legal entity")'
        )
    prompt = template
    print(f"Finding competitors of {company}...")

    try:
        response = run_chatgpt(prompt, model = 'gpt-4o')
        parsed_results.append(response.choices[0].message.content)

    except Exception as e:
        print(f"Error processing chunk {i}: {e}")
        parsed_results.append("")  # Append an empty string if there's an error

    initial_result = "\n".join(parsed_results)

    return initial_result

def competitor_search_workflow():
    result = find_competitors()
    
    # Display found competitors
    for i, link in enumerate(result['Website']):
        print(f"{i}: {link}")
        
    # Ask the user for additional competitors
    user_input = input(
        "\nThese are the competitors DumbResearch found. "
        "Do you have any additional players you would like to add to the search? "
        "Please add their respective websites, separated by a comma:\n"
    )
    add_players = [str(x.strip().lower().rstrip("/")) for x in user_input.split(",") if x.strip()]
    
    # Add new competitors to the result DataFrame
    for i in add_players:
        new_row = {'Competitor': None, 'Industry': None, 'Website': i, 'Legal entity': None}
        result = pd.concat([result, pd.DataFrame([new_row])], ignore_index=True)
    
    # Display the updated list of competitors
    for i, link in enumerate(result['Website']):
        print(f"{i}: {link}")
    
    # Ask the user to select competitors to keep
    user_input = input(
        "\nThis is the current set of competitors. "
        "Select the indexes of competitors you want to keep, separated by commas:\n> "
    )
    
    try:
        # Convert the input string into a list of integers
        indexes = [int(x.strip()) for x in user_input.split(",") if x.strip().isdigit()]
    except ValueError:
        print("Invalid input. Please enter a comma-separated list of numbers.")
        return {}
    
    # Validate indexes and filter the selected competitors
    chosen_competitors = result.iloc[indexes].reset_index(drop=True) if indexes else pd.DataFrame()
    
    return chosen_competitors

def search_research_workflow():
    competitor_set = competitor_search_workflow()
    print(f"Now looking at the websites of your competitor set {competitor_set['Website']}")
    output = dumbresearch_workflow(competitor_set['Website'])
    return output



import datetime
import pandas as pd

def parse_semicolon_csv_to_dataframe(csv_text):
    """
    Dynamically determine the max number of columns across all lines
    and use that to build the DataFrame.
    """
    lines = [line.strip() for line in csv_text.strip().splitlines() if line.strip()]
    if not lines:
        return pd.DataFrame()

    # Split everything first
    splitted_lines = [row.split(";") for row in lines]

    # Find the maximum column count
    max_cols = max(len(sl) for sl in splitted_lines)

    # First row is still used as headers, but it might not have all columns
    headers = splitted_lines[0]
    if len(headers) < max_cols:
        # Pad the header row with "ColumnX" or "empty" for missing columns
        missing_count = max_cols - len(headers)
        headers += [f"ExtraCol{i+1}" for i in range(missing_count)]
    elif len(headers) > max_cols:
        # If the header has more columns than any data row, just truncate it
        headers = headers[:max_cols]

    # For all subsequent rows, pad or truncate
    data_rows = []
    for row in splitted_lines[1:]:
        if len(row) < max_cols:
            row += ["empty"] * (max_cols - len(row))
        elif len(row) > max_cols:
            row = row[:max_cols]
        data_rows.append(row)

    df = pd.DataFrame(data_rows, columns=headers)
    return df


#############################################
# 2) Individual Extraction Functions
#############################################

def extract_income_statement(report_text):
    """
    Extracts the Income Statement from the given `report_text`.
    Returns a DataFrame, or empty DataFrame if none found.
    """
    # Single-task instructions to ChatGPT
    income_statement_instructions = """
    You are tasked with extracting only the Income Statement data
    from the following report. The output should be formatted as
    a semicolon-delimited table with a header row.

    Instructions:
    1. Only output the Income Statement (no other tables).
    2. Use semicolons (;) to separate columns.
    3. If certain cells are empty, write 'empty'.
    4. Do not include extra commentary or text outside the table.
    5. The first line should be the header row.

    Here is the input:
    {report_text}
    """

    prompt = income_statement_instructions.format(report_text=report_text)
    response_text = run_chatgpt(prompt)
    response_text = response_text.choices[0].message.content
    df = parse_semicolon_csv_to_dataframe(response_text)
    return df


def extract_operational_kpis(report_text):
    """
    Extracts important operational KPIs from the report text,
    such as production volume, customer count, churn rate, ARPU, etc.
    Returns a DataFrame.
    """
    kpi_instructions = """
    You are tasked with extracting only the operational KPIs from
    the following report. Examples of such KPIs include:
    - Number of customers or clients
    - Churn rate
    - ARPU (Average Revenue per User)
    - Employee headcount
    - Production volume
    - Utilization rates
    - Other similar quantitative metrics

    Output a semicolon-delimited table with a header row.
    Each row should contain:
       - KPI Name
       - KPI Value
       - (Optionally) a KPI Description

    Only output this single table, no other text.

    Here is the input:
    {report_text}
    """

    prompt = kpi_instructions.format(report_text=report_text)
    response_text = run_chatgpt(prompt)
    response_text = response_text.choices[0].message.content
    df = parse_semicolon_csv_to_dataframe(response_text)
    return df


def extract_qualitative_info(report_text):
    """
    Extracts only qualitative information, e.g.,
    management commentary, forward-looking statements, strategic objectives, etc.
    Returns a DataFrame with two columns: label, content.
    """
    qualitative_instructions = """
    You are tasked with extracting only qualitative information
    such as management commentary, forward-looking statements,
    strategic objectives, or other narrative insights from the
    following report. Each piece of qualitative info should be
    categorized under a 'label', with the actual commentary in 'content'.

    Output a semicolon-delimited table with this header:
    "label";"content"

    Only output this single table, no extra text or lines.

    Here is the input:
    {report_text}
    """

    prompt = qualitative_instructions.format(report_text=report_text)
    response_text = run_chatgpt(prompt)
    response_text = response_text.choices[0].message.content
    df = parse_semicolon_csv_to_dataframe(response_text)
    return df


#############################################
# 3) Orchestrator for a Single Report
#############################################

def parse_single_report(report_data):
    """
    Given one report (dictionary with keys like date, company, report),
    runs the three extraction functions and returns a dictionary of DataFrames.
    e.g. {
        "Income Statement": <DataFrame>,
        "KPIs": <DataFrame>,
        "Qualitative": <DataFrame>
    }
    """

    report_text = report_data.get("report", "")
    if not report_text:
        # Return empty placeholders if no text
        return {
            "Income Statement": pd.DataFrame(),
            "KPIs": pd.DataFrame(),
            "Qualitative": pd.DataFrame()
        }

    # Calls each extraction function separately
    income_df = extract_income_statement(report_text)
    kpi_df = extract_operational_kpis(report_text)
    qual_df = extract_qualitative_info(report_text)

    return {
        "Income Statement": income_df,
        "KPIs": kpi_df,
        "Qualitative": qual_df
    }


#############################################
# 4) parse_bundesanzeiger_reports
#############################################

def parse_bundesanzeiger_reports(reports_dict):
    """
    Takes a dictionary of reports, keyed by unique IDs, where each value 
    is a dictionary containing 'date', 'company', 'report' text, etc.

    Returns a final structure:
    {
        <company_name>: {
            <year>: {
                <report_id>: {
                   "Income Statement": DataFrame,
                   "KPIs": DataFrame,
                   "Qualitative": DataFrame
                },
                ...
            },
            ...
        },
        ...
    }
    """

    master_outputs = {}

    for report_id, report_data in reports_dict.items():
        # Identify the company
        company_name = report_data.get("company", "Unknown Company")

        # Parse out the year from the date
        raw_date = report_data.get("date", None)
        parsed_year = None
        if isinstance(raw_date, datetime.date):
            parsed_year = raw_date.year
        elif isinstance(raw_date, str):
            try:
                dt = datetime.datetime.strptime(raw_date, "%Y-%m-%d")
                parsed_year = dt.year
            except ValueError:
                print(f"Could not parse date for report {report_id}: {raw_date}")
        if not parsed_year:
            print(f"Skipping report {report_id}: No valid year.")
            continue

        # Build structure: master_outputs[company][year][report_id]
        if company_name not in master_outputs:
            master_outputs[company_name] = {}
        if parsed_year not in master_outputs[company_name]:
            master_outputs[company_name][parsed_year] = {}

        # Parse the single report using the orchestrator function
        results_for_this_report = parse_single_report(report_data)

        # Save the results
        master_outputs[company_name][parsed_year][report_id] = results_for_this_report

    return master_outputs

In [839]:
def get_bundesanzeiger_reports(companies):
    """
    companies: A DataFrame with a 'Legal entity' column, listing company names.
    Returns: A dict {company_name: [report_dict, ...]}
    """
    company_names = list(companies)
    ba = Bundesanzeiger()
    company_reports = {}
    for company in company_names:
        data = ba.get_reports(company)
        company_reports[company] = data
    return company_reports

def parse_bundesanzeiger_reports(reports_dict):
    """
    Takes a dictionary of reports, keyed by unique IDs, where each value
    is a dictionary containing 'date', 'company', 'report' text, etc.

    Returns a final structure:
    {
        <company_name>: {
            <year>: {
                <report_id>: {
                   "Income Statement": DataFrame,
                   "KPIs": DataFrame,
                   "Qualitative": DataFrame
                },
                ...
            },
            ...
        },
        ...
    }
    """
    master_outputs = {}
    for report_id, report_data in reports_dict[company].items():
        company_name = report_data.get("company", "Unknown Company")

        raw_date = report_data.get("date", None)
        parsed_year = None
        if isinstance(raw_date, datetime.date):
            parsed_year = raw_date.year
        elif isinstance(raw_date, str):
            try:
                dt = datetime.datetime.strptime(raw_date, "%Y-%m-%d")
                parsed_year = dt.year
            except ValueError:
                print(f"Could not parse date for report {report_id}: {raw_date}")
        if not parsed_year:
            print(f"Skipping report {report_id}: No valid year.")
            continue

        if company_name not in master_outputs:
            master_outputs[company_name] = {}
        if parsed_year not in master_outputs[company_name]:
            master_outputs[company_name][parsed_year] = {}

        # Parse the single report using the orchestrator
        results_for_this_report = parse_single_report(report_data)
        master_outputs[company_name][parsed_year][report_id] = results_for_this_report

    return master_outputs


In [835]:
reports = {'Myra Security GmbH':result}

In [893]:
for company in reports:
    for report_id, report_data in reports[company].items():
        company_name = report_data.get("company", "Unknown Company")

        raw_date = report_data.get("date", None)
        parsed_year = None
        if isinstance(raw_date, datetime.date):
            parsed_year = raw_date.year
        elif isinstance(raw_date, str):
            try:
                dt = datetime.datetime.strptime(raw_date, "%Y-%m-%d")
                parsed_year = dt.year
            except ValueError:
                print(f"Could not parse date for report {report_id}: {raw_date}")
        if not parsed_year:
            print(f"Skipping report {report_id}: No valid year.")
            continue

In [903]:
combined_reports = {}
unique_id_counter = 0

for company_name, reports_list in reports.items():
    for key,rpt in reports_list.items():
        unique_id = f"{company_name}_{unique_id_counter}"
        combined_reports[unique_id] = rpt
        unique_id_counter += 1

In [905]:
domain = 'https://www.greendc.ch'
links, content = get_all_links(domain)

In [824]:
company_name = 'Corero Network security plc'

os.environ["INFOGREFFE_API_KEY"] = "49013efb9c592cbdcf9cb109d60f8a2fe0210a8b42fc0a67145964cb"

api_key = os.getenv("INFOGREFFE_API_KEY")

result = 
base_url = 'https://opendata.datainfogreffe.fr/explore/dataset/chiffres-cles-2022/api/'

In [825]:
result

{'company_name': 'Corero Network security plc',
 'company_number': '02662978',
 'filing_history': [{'transaction_id': 'MzQ0NjI0MjgxMmFkaXF6a2N4',
   'barcode': 'XDHCQ2W0',
   'type': 'CS01',
   'date': '2024-12-05',
   'category': 'confirmation-statement',
   'description': 'confirmation-statement-with-no-updates',
   'description_values': {'made_up_date': '2024-11-24'},
   'pages': 3,
   'action_date': '2024-11-24',
   'links': {'self': '/company/02662978/filing-history/MzQ0NjI0MjgxMmFkaXF6a2N4',
    'document_metadata': 'https://document-api.company-information.service.gov.uk/document/7nPqvgqvM4Ygi6lynUt0eMTCl-jLMkvyHoN7Aq0dHQs'}},
  {'transaction_id': 'MzQ0MDI4NzM4OGFkaXF6a2N4',
   'barcode': 'XDE6T9S0',
   'type': 'SH01',
   'date': '2024-10-21',
   'category': 'capital',
   'description': 'capital-allotment-shares',
   'description_values': {'date': '2024-10-11',
    'capital': [{'currency': 'GBP', 'figure': '5,121,651.34'}]},
   'pages': 3,
   'action_date': '2024-10-11',
   'lin

In [271]:
clean_content

"Web DDoS Protection - Link11\nSolutions\nNetwork Security\nInfrastructure DDoS Protection\nCloud Insights\nWeb Protection\nWeb DDoS Protection\nBot Management\nZero Touch WAF\nAPI Protection\nWeb Performance\nSecure CDN\nSecure DNS\nIndustries\nHosting\nLogistic\nBanking & Finance\nE-Commerce\nUtilities\nOnline Gaming\nMedia\nPublic Sector\nBenchmark\nCustomers\nResources\nBlog\nGlossary\nPress\nDownloads\nVideos & Podcasts\nEvents\nAbout Link11\nCompany\nPatented DDoS Protection\nPartner\nCareer\nCarbon Neutrality\nContact\nen\nde\nLogin\nUnder attack?\nContact\nWeb DDoS Protection\nCloud-based DDoS protection for web applications\nAutomation ensures 24/7 protection\nZero time to mitigate for known,\n<\n10 seconds for new vectors\n4.3/5 Google Reviews\nBook a demo\nLearn more\nPrecise detection\nNo need to worry about complex attacks anymore: The system detects traffic anomalies and protects you from emerging threats in real time.\nFull automation\nNo work for you: manual operation i

# User pastes in links they want to look at. Programm takes links and asks 
* which part of the website the user wants to look at
* for each of the subsection what they want the output to look like and a description
* where they want the output to be saved

# Program runs for each website, creates the tabular output for each of the websites subsections (e.g. product, locations)

# Tabular output is saved to the specified location

Appendix

parse_description = ('These are the solutions offered by a german cyber security player (input is provided in German), please provide a table or tables that list the respective products / solutions with important information on the respective products e.g. their descriptions, features, target customers, KPIs')

#Function to extract deeper-level headlines
def extract_headlines_with_depth(links, depth):
    headlines = set()
    for link in links:
        try:
            html = run_selenium(domain)
            time.sleep(10)

            soup = BeautifulSoup(html, 'html.parser')
            
            for a_tag in soup.find_all('a', href=True):
                # Resolve relative URLs
                full_url = urljoin(link, a_tag['href'])
                parsed = urlparse(full_url)
                path_parts = parsed.path.strip('/').split('/')
                if len(path_parts) == depth:
                    headlines.append(full_url)

        except Exception as e:
            print(f"Error visiting {full_url}: {e}")
            parsed = urlparse(link)
            path_parts = parsed.path.strip('/').split('/')
            if len(path_parts) == depth:
                headline = "/".join(path_parts[:depth])
                headlines.add(headline)
    return sorted(headlines)


def extract_second_level_headlines(links):
    # Parse URLs and extract paths
    headlines = set()  # Use a set to avoid duplicates
    for link in links:
        parsed = urlparse(link)
        path_parts = parsed.path.strip('/').split('/')

        # Match second-level paths (e.g., /solutions/, /locations/)
        headline = "/".join(path_parts[:1])
        headlines.add(headline)

    return sorted(headlines)
    
def get_select_data(headlines, domain,depth = 3):
    domain = domain
    subpage_links = set(get_all_links(domain))  # Assuming this fetches all sub-links from the domain
    
    # Dictionary to store concatenated text for each key
    content_dict = defaultdict(str)

    # Fetch and process links
    urls = {}
    for page in headlines:
        # Match URLs for the current page
        urls[page] = re_match_second_level_headlines(page, domain, subpage_links)
        for url in urls[page]:  # Iterate over the URLs for the current page
            if not is_valid_url(url):  # Validate URL before scraping
                print(f"Invalid URL skipped: {url}")
                continue

            print(f"Scraping URL: {url}")
            result = run_selenium(url)  # Scrape website content
            body_content = extract_body_content(result)  # Extract body content
            cleaned_content = clean_body_content(body_content)  # Clean the content

            # Concatenate the cleaned content to the respective key in the dictionary
            content_dict[page] += cleaned_content + " "  # Add a space between concatenated content

    return content_dict
    
    

def re_match_second_level_headlines(headline, domain, links_to_visit):
    matching_urls = []
    headline_url = urljoin(domain, headline)
    matching_urls.extend([url for url in links_to_visit if url.startswith(headline_url)])
    return matching_urls
    
    
        refinement_prompt = (f'This is the output from an LLM that has been fed multiple chunks of information: {initial_result}.'
                     'Please clean this output up by following these instructions carefully: \n\n'
                     '1. Format: The output will likely be in multiple tables, please condense all output into one table. IMPORTANT: “Output the table as valid CSV with exactly the same number of columns for all rows (have fill value "empty" if there is not input for a given cell), no extra lines, and properly quoted cells if they contain commas. Use a semicolon to seperate cells instead of a comma”'
                     '2. No duplication: The table you create should not contain duplicate rows, there should only be one row with relevant information for each e.g. product, location, customer etc'
                     '3. If the tables you receive have multiple different categories (e.g. solutions, locations, customers) still include them into one table and add a columns called "type" to your table that captures this information'
                     )






def parse_with_chatgpt(dom_chunks, parse_description):
    """
    Sends chunks of website content to OpenAI's GPT API for parsing based on a description.
    """

    template = (
        f'You are tasked with extracting specific information from the following text content: {dom_chunks}.'
        'Please follow these instructions carefully: \n\n'
        f' 1. ** Extract Information** : Only extract the information that directly matches the provided description {parse_description}'
        ' 2. ** No Extra Content** : Do not include any additional text, comments or explanations in your response.'
        ' 3. ** Empty Response** : If no information matches the description return to an empty string'
        ' 4. ** Direct Data** : Your response should contain only data that is explicitly requested, with no other text.'
        ' 5. ** Format** : **Single Table**: Output exactly one table, no extra lines or text.'
                        '  - Use semicolons (`;`) to separate columns.'
                        '  - Properly quote cells with ("") if they contain semicolons or commas.'
                        '  - If a cell has no data, use `empty`.'
        )

    # Loop through chunks and get responses
    parsed_results = []
    for i, chunk in enumerate(dom_chunks, start=1):
        prompt = templa
te.format(dom_chunks=chunk, parse_description=parse_description)
        print(f"Processing chunk {i}/{len(dom_chunks)}...")

        try:
            response = run_chatgpt(prompt)
            
            if isinstance(response, list):
                parsed_results.append(response[0].choices[0].message.content)
            else:
                parsed_results.append(response.choices[0].message.content)

            # Extract the content of the assistant's reply
            parsed_results.append(response[0].choices[0].message.content)

        except Exception as e:
            print(f"Error processing chunk {i}: {e}")
            parsed_results.append("")  # Append an empty string if there's an error

    final_result = []

    initial_result = "\n".join(parsed_results)

    refinement_prompt = (f'This is the output from an LLM that has been fed multiple chunks of information: {initial_result}. Please merge them into a single CSV table with these requirements:'
                        '1. **Single Table**: Output exactly one table, no extra lines or text.'
                        '2. **Format**:'
                        '   - Use semicolons (`;`) to separate columns.'
                        '   - Properly quote cells with ("") if they contain semicolons or commas.'
                        '   - If a cell has no data, use `empty`.'
                        '3. **Columns**:'

                        '   - All rows must have the same number of columns.'
                        '   - No duplicate rows.'
                        '   - If the data covers multiple categories (e.g. solutions, locations, customers), add a column named `"Type"` to capture the category.'
                        '4. **No Duplicates**: Merge repeated rows or near-identical information into a single row.'
                        '5. **Final Output**:'
                        '   - No code fences or markdown.'
                        '   - Strictly the CSV text (with one header row).')

    if len(initial_result) > 12000:
        dom_chunks = split_dom_content(initial_result)
        for i, chunk in enumerate(dom_chunks,start=1):
            prompt_02 = refinement_prompt.format(initial_result=chunk)
            print(f"Processing refinement chunk {i}/{len(dom_chunks)}...")
            try:
                response = run_chatgpt(prompt_02)
                if isinstance(response, list):
                    final_result.append(response[0].choices[0].message.content)
                else:
                    final_result.append(response.choices[0].message.content)

                # Extract the content of the assistant's reply
                final_result.append(response[0].choices[0].message.content)

            except Exception as e:
                print(f"Error processing refinement chunk {i}: {e}")
                final_result.append("")  # Append an empty string if there's an error

    else:

        prompt_02 = refinement_prompt.format(initial_result=initial_result)
        print(f"Refining initial result...")
        try:
            response = run_chatgpt(prompt_02)
            if isinstance(response, list):
                final_result.append(response[0].choices[0].message.content)
            else:
                final_result.append(response.choices[0].message.content)

            # Extract the content of the assistant's reply
            final_result.append(response[0].choices[0].message.content)

        except Exception as e:
            print(f"Error processing chunk {i}: {e}")
            final_result.append("")  # Append an empty string if there's an error

    return "\n".join(final_result)

    def filter_urls_by_stem(stem_url, url_list):
    """
    Compares each URL in url_list to stem_url.
    If a URL starts with the stem_url, it's added to a matches list.
    
    :param stem_url: A string representing the 'stem' portion of the URL.
    :param url_list: A list of full URLs to check against the stem URL.
    :return: A list of URLs that match the stem URL.
    """
    matches = []
    for random_url in url_list:
        # Simple check: does the random_url begin with stem_url?
        if random_url.startswith(stem_url) and random_url not in matches and random_url != stem_url:
            matches.append(random_url)
    return matches

                     